Перед началом прохождения убедитесь, что все настроено как надо. Во-первых, **перезапустите ядро** (**restart the kernel**) (в панели меню выберите Kernel$\rightarrow$Restart) и затем **запуск всех ячеек** (**run all cells**) (в панели меню выберите Cell$\rightarrow$Run All).

Убедитесь, что вы заменили всем такие места 
```python
# YOUR CODE HERE
raise NotImplementedError()
```
и
```
"YOUR ANSWER HERE"
```
на свои решения.

> Не переименовывайте ноутбук!

Также, заполните свои данные (имя и группу):

In [1]:
NAME = "Вячеслав"
GROUP = "6492"

In [2]:
import uuid

STUDENT_UUID = uuid.uuid4()
print(f"Your uuid is {STUDENT_UUID}")

Your uuid is 6ea7b1cb-7dc2-48ce-b7dc-dac965396d5f


---

# Приветствуем тебя, путник!

Это вторая контрольная работа с некоторыми вопросами по теме задачи регрессии в машинном обучении!

Если у вас не установлены какие-то пакеты, то посмотрите нужные версии в ноутбуках по регрессии (10-13).

Не забывайте о следующих особенностях:
- Помимо указанных тестов после задания существуют и скрытые тесты, поэтому внимательнее читайте задание
- Обязательно проверяйте выполнимость на тестах, для проверки перезапустите ядро и запустите все ячейки заново
- После скачивания ноутбуков достаточно заполнить места для ответов и отправить ноутбук обратно

In [3]:
import numpy as np
print(f"numpy version: {np.__version__}")
import pandas as pd
print(f"pandas version: {pd.__version__}")
# Just to check that it exists
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

from typing import Tuple, List

np.random.seed(123)

numpy version: 1.21.2
pandas version: 1.3.3
scikit-learn version: 0.24.2


### Задание 1

Реализуйте функцию загрузки данных. В качестве результата необходимо вернуть tuple с DataFrame признаков и Series целевой переменной. Используйте [эту функцию](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html) в качестве основы для загрузки датасета диабетов.

> Это очень простая задача, но ее важно выполнить, так как она будет использоваться далее по тестам!

In [4]:
from sklearn.datasets import load_diabetes
def load_data() -> Tuple[pd.DataFrame, pd.Series]:
    diabetes = sklearn.datasets.load_diabetes(as_frame=True)
    X = diabetes.data
    y = diabetes.target
    return X,y

In [5]:
X, y = load_data()

assert isinstance(X, pd.DataFrame)
assert isinstance(y, pd.Series)


### Задание 2

Реализуйте функцию поиска признаков в данных с корреляцией выше порога относительно указанной переменной.

Необходимо вернуть список имен переменных. Саму переменную необходимо исключить (так как корреляция относительно самого себя всегда = 1).

In [6]:
def get_high_correlation_features(df: pd.DataFrame, col_name: str, threshold: float) -> List[str]:
    df_corr = df.corr()
    mask = abs(df_corr[col_name]) > threshold
    res = []
    for i in range(len(mask)):
        if (mask[i] == True)&(mask.index[i] != col_name):
            res.append(mask.index[i])
    return res

In [7]:
X, y = load_data()
df = X.copy()
df['target'] = y

_test_data = get_high_correlation_features(df, "target", threshold=0.35)
np.testing.assert_equal(
    _test_data, 
    np.array(['bmi', 'bp', 's3', 's4', 's5', 's6'])
)


### Задание 3

Реализуйте класс для предобработки данных. Предобработка должна включать в себя:
- Масштабирование признаков к пределам [-1; 1]
- Определение 1го и 3го квартилей по указанному признаку и удаление данных за пределами квартилей по значениям признака. Удалить только данные больше 3го и меньше 1го квартилей (не равно).

> Обратите внимание, в каком порядке должны идти операции в transform(), взгляните на то, на каких данных училась модель предобработки в fit()

In [43]:
from sklearn.preprocessing import MinMaxScaler
class DataPreprocessing:
    def __init__(self, quartile_clean_col_name: str):
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.col = quartile_clean_col_name
        pass
        
    def fit(self, X: pd.DataFrame) -> None:
        df_copy = X.copy()
        quart1 = df_copy[self.col].quantile(0.25)
        quart3 = df_copy[self.col].quantile(0.75)
        print(quart1, quart3)
        mask = (df_copy[self.col] < quart1)|(df_copy[self.col] > quart3)
        self.df = df_copy[~mask]
        self.scaler = self.scaler.fit(self.df)
        print(np.shape(self.df))
        pass
        
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        column = self.df.columns
        index = self.df.index
        df = self.scaler.transform(self.df)
        df=pd.DataFrame(data=df,index=index,columns=column)
        return df

In [44]:
X, y = load_data()

_X_train_part = X.iloc[:300]
_X_test_part = X.iloc[300:]

preprocess = DataPreprocessing(quartile_clean_col_name='s2')
preprocess.fit(_X_train_part)
_X_test_prep = preprocess.transform(_X_test_part)

assert isinstance(_X_test_prep, pd.DataFrame)
np.testing.assert_equal(_X_test_prep.shape, (71, 10))
assert _X_test_prep["sex"].min() < -0.95
assert _X_test_prep["s5"].max() > 0.95


-0.032707010720619274 0.02655633567304975
(150, 10)


AssertionError: 
Items are not equal:
item=0

 ACTUAL: 150
 DESIRED: 71

### Задание 4

Реализуйте класс для обучения модели линейной регрессии с вычислением метрик. результирующие ключи метрик смотрите по тестам. В метрики должны входить: MSE, RMSE, MAE, R^2.

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
class ModelTrainWithMetrics:
    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        self.model = LinearRegression().fit(X, y)

    def predict_evaluate(self, X: pd.DataFrame, y: pd.Series)-> Tuple[np.ndarray, dict]:
        y_pred = self.model.predict(X)

        metrics = {
        "mse": mean_squared_error(y, y_pred),
        "mae": mean_absolute_error(y, y_pred),
        "rmse": mean_squared_error(y, y_pred, squared=False),
        "r2": r2_score(y, y_pred)
        }
        return tuple((y_pred, metrics))


In [42]:
from sklearn.model_selection import train_test_split

X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.3)

model = ModelTrainWithMetrics()
model.fit(X_train, y_train)
y_test_pred, metrics = model.predict_evaluate(X_test, y_test)

assert isinstance(metrics, dict)
assert isinstance(y_test_pred, np.ndarray)
np.testing.assert_equal(set(metrics.keys()), set(['mse', 'mae', 'rmse', 'r2']))

assert metrics['r2'] > 0.45
assert metrics['rmse'] < 55
assert metrics['mae'] < 45


### Задание 5

Реализуйте функцию сбора статистики данных датасета по каждой числовой колонке.

Формат выхода: колонки ~ показатели, строки ~ названия исходных колонок.

Требуемые показатели:
- Минимальное значение колонки
- Максимальное значение колонки
- Медианное значение колонки
- Среднее значение колонки / Mean value
- Стандартное (Среднеквадратическое) отклонение / Standart deviation
- Первый квартиль
- Третий квартиль
- Интерквартильный размах (IQR ~ Interquartile range)

In [95]:
def collect_statistics(df: pd.DataFrame) -> pd.DataFrame:
    columns = ['min', 'max', 'median', 'mean', 'std', 'Q1', 'Q3', 'IQR']
    index = df.transpose().index
    arr = []
    shape = np.shape(df)
    arr.append(df.min())
    arr.append(df.max())
    arr.append(df.median())
    arr.append(df.mean())
    arr.append(df.std())
    arr.append(df.quantile(0.25))
    arr.append(df.quantile(0.75))
    arr.append(df.quantile(0.75)-df.quantile(0.25))
    df = pd.DataFrame(data=np.asarray(arr).transpose(), index=index, columns=columns)
    return(df)
    
   

In [96]:
X, _ = load_data()

stats = collect_statistics(X)

np.testing.assert_equal(set(stats.index), set(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']))
np.testing.assert_equal(set(stats.columns), set(['min', 'max', 'median', 'mean', 'std', 'Q1', 'Q3', 'IQR']))

np.testing.assert_almost_equal(stats.loc[['age', 'bmi'], 'mean'], [0, 0], decimal=2)
np.testing.assert_almost_equal(stats.loc[['s3', 'bp'], 'std'], [0.048, 0.048], decimal=2)
np.testing.assert_almost_equal(stats.loc[['s6', 's5'], 'min'], [-0.138, -0.126], decimal=2)
np.testing.assert_almost_equal(stats.loc[['s1', 's2'], 'max'], [0.154, 0.199], decimal=2)


### Задание 6 (открытый вопрос)

Что такое нормализация данных? Почему для линейных моделей так важна нормализация данных (1 балл)?

Что такое интерпретируемость весов? Приведите пример(-ы) для объяснения (1 балл).

> Если вы пишите ответ на вопрос, но не приводите пример, то балл не засчитывается.

> Для примеров кода можно использовать конструкцию внутри markdown ячеек:

```python
def function_sample():
    pass
```

Или создавать ячейки с кодом и писать в них.

Нормализация данных - это изменение масштаба данных из исходного диапазона, чтобы все значения находились в диапазоне от 0 до 1. Нормализация необходима, когда данные имеют входные значения разного масштаба и если их не нормализовать, то анализ данных может быть не верным. Для нормализации мы использовали MinMaxScaler()

Интерпретируемость весов - это когда по весам можно обозначить более важные показатели. Мы рассматривали задачу, в которой при неправильном масштабировании выходило, что посещаемость важнее выполненных заданий. После масштабирования все вернулось на круги своя и веса начали показывать правильные показатели, показывающие реальную картину важности показателей. Масштабирование также с помощью MinMaxScaler()

### Задание 7 (открытый вопрос)

Какие способы визуализации (визуального анализа) данных вы знаете? Приведите примеры (можно отобразить на загруженных данных из задач до этого) (1 балл).

Объясните, плюсы и минусы каждого (доп. балл).

Визуализация данных при помощи диаграммы рассеяния (plt.scatter(x,y)), линейные графики (ax.plot(x, y)), гистограммы (plt.hist(x, bins=50, density=1, color='g', alpha=0.75)), Bar plots (plt.bar(x_labels, group1_scores)), sns.displot(df_src['RM'], kde=True, height=7, aspect=1.5), sns.pairplot(df_cleaned[features], diag_kind='auto', height=6)

### Задание 8 (открытый вопрос)

Объясните эффекты "недообучения" и "переобучения". Приведите примеры и объясните, как их определить (1 балл).

Расскажите о способах борьбы с каждым (1 балл).

Недообучение - это когда модель слишком простая и не полностью описывает объект, так как данные являются неточными и восстановить зависимость не получится. Например, мы брали данные, у которых полиномиальная зависимость и модель можно было описать даже моделью второго порядка. Но с недообучением мы получали линию, проходящую через точки. ПРи этом показатель R2 является низким и по нему можно определить недообучение.

Переобучение - это когда модель является слишком сложной. При этом ошибка обучения уменьшается, но ошибка проверки начинает расти. Определить можно с помощью абсолютной средней ошибке (mae), сравнив показатели train и test выборок.

Чтобы бороться с данными эффектами необходимо искать оптимальную модель. Это можно делать ориентируясь на показатели, а также визуализируя данные, чтобы увидеть график выборки и полученной модели. Также, можно попробовать увеличить объем обучающей выборки, так как переобучение может возникнуть из-за недостатка данных.